外部資料

In [1]:
import pandas as pd
import numpy as np
from glob import glob, iglob

df = pd.DataFrame()

# iglob使用較少memory
for csv in iglob('../0821/*_top500_clean.csv'):
    tmp_df = pd.read_csv(csv)
    df = df.append(tmp_df, ignore_index=True)

內部資料(飯店經緯度)

In [2]:
df_gps = pd.read_csv("F:\\NCTU\\lab\\奧丁丁\\奧丁丁資料前處理\\OwlTing_整合資料\\csv\\data_gps.csv")

總共1500多家飯店

In [40]:
df_gps.shape

(1565, 5)

內部資料(飯店名稱)

In [27]:
df_name = pd.read_csv("F:\\NCTU\\lab\\奧丁丁\\奧丁丁資料前處理\\OwlTing_整合資料\\csv\\data.csv")
df_name = df_name[['hotel_id', 'name']].copy()

結合內部資料

In [35]:
len(df_name.name.value_counts()) # 只有900家飯店知道名稱

889

In [39]:
df_name.drop_duplicates()

,hotel_id,name
0,166,風車花園城堡民宿
36,167,夢想森林親子民宿(觀光局百大好客民宿)
1573,168,可以住
1939,169,台悠丸‧作夥憩海邊
1940,170,陋室影像民宿
...,...,...
573995,2125,Hin Loi Guesthouse
574088,2126,Skypod Hostel
574283,2128,花巷650
574319,2132,CasaBiancaVilla


In [3]:
df_gps.latitude.isna().sum()

73

緯度為null

In [7]:
df_gps.longitude.isna().sum()

232

經度為0

In [9]:
len(df_gps.loc[df_gps.latitude == 0])

364

In [11]:
len(df_gps.loc[df_gps.longitude == 0])

200

移除經緯度為0或是null的飯店

In [3]:
df_gps_clean = df_gps.dropna(subset=['latitude', 'longitude']) # 僅刪除經緯度有0的

In [4]:
df_gps_clean = df_gps_clean[df_gps_clean.latitude!=0]
df_gps_clean = df_gps_clean[df_gps_clean.longitude!=0]

In [18]:
df_gps_clean.isna().sum()

hotel_id       0
country      196
city         339
latitude       0
longitude      0
dtype: int64

In [5]:
def get_first(x, length):
    return x[:length]

# 把經緯度成字串保留想要的位數
df['lat'] = df['lat'].astype('str').apply(get_first, args=(8,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(10,)).astype('float')

內外部資料都只用小數點4位看能不能和

In [6]:
def get_first(x, length):
    return x[:length]

# 把經緯度成字串保留想要的位數
df['lat'] = df['lat'].astype('str').apply(get_first, args=(6,)).astype('float')
df['lng'] = df['lng'].astype('str').apply(get_first, args=(7,)).astype('float')
df_gps_clean['latitude'] = df_gps_clean['latitude'].astype('str').apply(get_first, args=(6,)).astype('float')
df_gps_clean['longitude'] = df_gps_clean['longitude'].astype('str').apply(get_first, args=(7,)).astype('float')

找有沒有經緯度完全重複的

In [7]:
df.duplicated(subset=['lat', 'lng'], keep=False).sum()

1659

In [41]:
df.duplicated(subset=['lat', 'lng']).sum()

1060

In [47]:
df_gps_clean.duplicated(subset=['latitude', 'longitude'], keep=False).sum()

39

經緯度同時重疊

In [44]:
df[df.duplicated(subset=['lat', 'lng'], keep=False)].sort_values(by=['lat']).sort_values(by=['lng'])

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count
764,麗馨精品商旅七賢館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.618,120.268,前金區市中一路229號,3.0,高雄,前金,"NT$817 - NT$2,169",4.0,18.0
574,奇異果快捷旅店-九如店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.618,120.268,三民區九如一路790號,3.0,高雄,三民,"NT$849 - NT$2,012",4.0,155.0
535,巴黎商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.625,120.280,新興區自立二路8號,3.0,高雄,新興,"NT$1,257 - NT$2,075",3.5,31.0
643,芳橙汽車旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.625,120.280,小港區中安路616號,3.0,高雄,小港,"NT$1,069 - NT$1,509",3.0,1.0
745,六合轉角737館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,22.622,120.283,南台路六合夜市 - 美麗島捷運站旁,2.0,高雄,新興,"NT$849 - NT$1,886",4.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...
1398,金色年代旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,25.121,121.861,板橋區重慶路66號12樓,3.0,新北,板橋,"NT$1,257 - NT$2,798",4.0,10.0
3661,大里驛青年旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.966,121.922,頭城鎮濱海路6段317-2號,2.0,宜蘭,頭城,"NT$1,980 - NT$3,018",5.0,16.0
3943,我家商務旅店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.966,121.922,羅東鎮中正路186之2號 3樓,2.5,宜蘭,羅東,"NT$1,100 - NT$3,018",0.0,0.0
1461,九份聽山,https://www.tripadvisor.com.tw/Hotel_Review-g1...,25.015,121.944,瑞芳區基山街219之4號,3.0,新北,瑞芳,"NT$943 - NT$2,703",3.0,11.0


In [19]:
df[df.duplicated(subset=['lat', 'lng'], keep=False)].groupby(['lat', 'lng']).agg('count')

title  uri  hotel_address  hotel_star  hotel_city  \
lat    lng                                                          
21.912 120.847      2    2              2           2           2   
       120.848      2    2              2           2           2   
21.918 120.844      2    2              2           2           2   
21.920 120.844      2    2              2           2           2   
21.925 120.832      2    2              2           2           2   
...               ...  ...            ...         ...         ...   
25.168 121.445      3    3              3           3           3   
25.169 121.445      2    2              2           2           2   
25.171 121.447      4    4              4           4           4   
25.180 121.689      2    2              2           2           2   
25.255 121.511      2    2              2           2           2   

                hotel_section  price_range  avg_rating  comment_count  
lat    lng                                                             
21.912 120.847              2            2           2              2  
       120.848              2            2           2              2  
21.918 120.844              2            2           2              2  
21.920 120.844              2            2           2              2  
21.925 120.832              2            2           2              2  
...                       ...          ...         ...            ...  
25.168 121.445              3            3           3              3  
25.169 121.445              2            2           2              2  
25.171 121.447              4            4           4              4  
25.180 121.689              2            2           2              2  
25.255 121.511              2            2           2              2  

[599 rows x 9 columns]

經度重疊

In [32]:
df[df['lat'].isin(df['lat'][df['lat'].duplicated()])]

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count
3,嘉義優遊商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48274,120.463,西區中山路617號11樓,3.0,嘉義,東區,"NT$1,163 - NT$2,734",3.5,41.0
39,楓雅SPA 汽車旅館 (大雅館),https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48274,120.431,大雅路二段256號,3.5,嘉義,東區,"NT$1,194 - NT$1,697",4.0,1.0
45,香緹汽車旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48041,120.463,西區八德路2號,3.0,嘉義,西區,"NT$1,980 - NT$2,922",3.5,4.0
70,波士頓大飯店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.48041,120.463,東區中正路673號,3.0,嘉義,西區,"NT$691 - NT$1,508",3.0,11.0
72,雙鳳大旅社,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.99303,121.593,復興街5號,2.0,花蓮,花蓮市,"NT$597 - NT$2,766",3.5,11.0
...,...,...,...,...,...,...,...,...,...,...,...
4028,田野小徑,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.80044,121.756,礁溪鄉份尾一路3號,3.5,宜蘭,礁溪,"NT$2,075 - NT$3,395",4.0,1.0
4029,玫瑰莊園民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.67588,121.751,冬山鄉梅花路742號,3.5,宜蘭,冬山,"NT$1,980 - NT$4,527",0.0,0.0
4030,夜市橙堡渡假旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.82802,121.774,新民路19號,3.5,宜蘭,宜蘭市,"NT$1,320 - NT$2,515",4.0,12.0
4033,賓成精品商旅,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.67275,121.809,西後街26-1號,3.0,宜蘭,宜蘭市,"NT$1,163 - NT$2,389",3.0,9.0


嘗試能不能concat

In [9]:
pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']).head()

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,hotel_id,country,city,latitude,longitude
0,花蓮北吉光輕旅青年旅館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.599,花蓮市中山路601巷1弄19號,2.0,花蓮,花蓮市,"NT$817 - NT$1,949",5.0,432.0,1482,台灣,花蓮市,23.988,121.599
1,希臘仙境美宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.599,花蓮市北濱街102-2號,3.0,花蓮,花蓮市,"NT$1,603 - NT$3,018",5.0,1.0,1482,台灣,花蓮市,23.988,121.599
2,美麗晨曦民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.599,吉安鄉太昌村明義二街16號,3.0,花蓮,吉安,"NT$2,483 - NT$4,244",5.0,235.0,1482,台灣,花蓮市,23.988,121.599
3,海洋飯店,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.604,花蓮市國聯三路58號,3.0,花蓮,花蓮市,"NT$1,509 - NT$2,043",4.0,1.0,1297,台灣,NaN,23.990,121.604
4,太魯閣蘇西小空間,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.604,新城鄉新城村中正路20號,2.5,花蓮,新城,"NT$1,069 - NT$2,672",3.5,14.0,1297,台灣,NaN,23.990,121.604


能和的飯店數目

In [10]:
len(pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']))

759

檢查是否有同一間飯店被重複

In [24]:
pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude']).title.value_counts()

花蓮樂活亞綠亞民宿    4
12號民宿        4
鐵道旅人民宿       4
愛格斯民宿        4
微笑拉菲草        4
            ..
墾丁一號旅店       1
隆德旅客之家       1
嘉蓮小棧         1
小羊房民宿        1
台北國際飯店       1
Name: title, Length: 568, dtype: int64

In [25]:
df_tmp = pd.merge(df, df_gps_clean, left_on=['lat', 'lng'], right_on=['latitude', 'longitude'])
df_tmp[df_tmp['title'].isin(df_tmp['title'][df_tmp['title'].duplicated()])]

,title,uri,lat,lng,hotel_address,hotel_star,hotel_city,hotel_section,price_range,avg_rating,comment_count,hotel_id,country,city,latitude,longitude
11,船長不在家旅舍,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.605,國聯3路85號,2.0,花蓮,花蓮市,"NT$786 - NT$2,578",4.5,3.0,795,TW,NaN,23.990,121.605
12,船長不在家旅舍,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.990,121.605,國聯3路85號,2.0,花蓮,花蓮市,"NT$786 - NT$2,578",4.5,3.0,796,台灣,NaN,23.990,121.605
13,踩風民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.602,吉安鄉福興村11鄰福興七街500號,3.0,花蓮,吉安,"NT$2,515 - NT$3,961",5.0,24.0,1078,台灣,NaN,23.988,121.602
14,踩風民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.602,吉安鄉福興村11鄰福興七街500號,3.0,花蓮,吉安,"NT$2,515 - NT$3,961",5.0,24.0,1283,台灣,花蓮縣,23.988,121.602
15,七星燈塔民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,23.988,121.602,新城鄉七星街59號,3.0,花蓮,花蓮市,"NT$2,389 - NT$3,835",4.5,9.0,1078,台灣,NaN,23.988,121.602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,海灘風情,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.650,121.736,頭城鎮外澳里濱海路二段68號,2.5,宜蘭,頭城,"NT$2,483 - NT$3,835",3.0,3.0,1172,台灣,宜蘭縣,24.650,121.736
730,香草熊河濱會館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.650,121.736,五結鄉二結路357巷39弄16號,3.5,宜蘭,五結,"NT$1,163 - NT$2,892",0.0,0.0,690,台灣,NaN,24.650,121.736
731,香草熊河濱會館,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.650,121.736,五結鄉二結路357巷39弄16號,3.5,宜蘭,五結,"NT$1,163 - NT$2,892",0.0,0.0,1172,台灣,宜蘭縣,24.650,121.736
742,神雕雅築庭園民宿,https://www.tripadvisor.com.tw/Hotel_Review-g1...,24.677,121.797,五結鄉福和二路118巷62號,3.5,宜蘭,五結,"NT$2,106 - NT$3,866",5.0,23.0,277,台灣,宜蘭縣,24.677,121.797
